this script deletes the necessary duplicate files found in final_metadata .xlsx

In [1]:
!pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [1]:
working_with_wav = True

In [2]:
import pandas as pd

files_to_delete = pd.read_excel("../../datasets/birdsongs-combined/metadata/final_metadata.xlsx", "FILES_TO_DELETE")["file to delete"].tolist()

if working_with_wav:
    files_to_delete = [x.replace("mp3", "wav").replace("ogg", "wav") for x in files_to_delete]

files_to_delete

['easmea/XC406586.wav',
 'louwat/XC136316.wav',
 'louwat/XC502062.wav',
 'prawar/XC376774.wav',
 'rebmer/XC475655.wav',
 'weskin/XC351819.wav',
 'aldfly/XC134874.wav',
 'aldfly/XC135454.wav',
 'aldfly/XC135455.wav',
 'aldfly/XC135456.wav',
 'aldfly/XC135457.wav',
 'aldfly/XC135459.wav',
 'aldfly/XC135460.wav',
 'aldfly/XC135883.wav',
 'aldfly/XC137570.wav',
 'aldfly/XC138639.wav',
 'aldfly/XC139577.wav',
 'aldfly/XC140298.wav',
 'aldfly/XC142065.wav',
 'aldfly/XC142066.wav',
 'aldfly/XC142067.wav',
 'aldfly/XC142068.wav',
 'aldfly/XC142329.wav',
 'aldfly/XC144672.wav',
 'aldfly/XC144674.wav',
 'aldfly/XC144675.wav',
 'aldfly/XC154310.wav',
 'aldfly/XC154449.wav',
 'aldfly/XC157462.wav',
 'aldfly/XC167789.wav',
 'aldfly/XC167791.wav',
 'aldfly/XC16967.wav',
 'aldfly/XC178480.wav',
 'aldfly/XC178481.wav',
 'aldfly/XC178485.wav',
 'aldfly/XC179417.wav',
 'aldfly/XC179600.wav',
 'aldfly/XC180091.wav',
 'aldfly/XC181484.wav',
 'aldfly/XC182735.wav',
 'aldfly/XC188377.wav',
 'aldfly/XC188432

In [10]:
len(files_to_delete)

15874

In [3]:
#train_audio_directory = "/mnt/lustre/peprmint/train_audio/"
train_audio_directory = "../../datasets/birdsongs-combined/train_audio/"

if working_with_wav: train_audio_directory = train_audio_directory[:-1] + "_wav/"

train_audio_directory

'/mnt/lustre/peprmint/train_audio_wav/'

In [5]:
import os

# attempts to delete every file from files_to_delete
# if the file cannot be found, it skips it
# if the file can be found, it deletes it

#for file in files_to_delete:

files_not_found = 0
files_deleted = 0

for file in files_to_delete:
    file_path = train_audio_directory + file

    try:
        os.remove(file_path)
        files_deleted += 1
    except OSError:
        files_not_found += 1
        pass

print(f"Out of the {len(files_to_delete)} files to delete, {files_deleted} were deleted. {files_not_found} file{'s' if files_not_found != 1 else ''} could not be found.")

Out of the 15874 files to delete, 15868 were deleted. 6 files could not be found.


In [7]:
# verify no duplicate XC ids are in the dataset

In [8]:
folders = os.listdir(train_audio_directory)

files = []
for folder in folders:
    files += [file for file in os.listdir(train_audio_directory + folder)]

all_ids = len(files)
unique_ids = len(set(files))

In [9]:
if all_ids == unique_ids:
    print("All duplicates removed and handled already.")
else:
    print(f"{all_ids - unique_ids} files are duplicates.")

    files = sorted(files)
    duplicate_files = []

    for i in range(len(files) - 1):
        j = i + 1

        if files[i] == files[j]:
            if files[i] not in duplicate_files:
                duplicate_files.append(files[i])

    for duplicate_file in duplicate_files:
        # identify the folder(s) the duplicate appears in.
        duplicates = []

        for folder in folders:
            folder_files = os.listdir(train_audio_directory + folder)

            if duplicate_file in folder_files:
                full_file_path = train_audio_directory + folder + "/" + duplicate_file

                date_modified = os.path.getmtime(full_file_path)
                duplicates.append([full_file_path, date_modified])

        duplicates = sorted(duplicates, key=lambda x: x[1], reverse=True)

        print(f"{duplicate_file}:    The newest file is {duplicates[0][0]}, thus all other files will be deleted.")

        duplicate_files_to_delete = duplicates[1:]

        for duplicate_file_to_delete in duplicate_files_to_delete:
            try:
                os.remove(duplicate_file_to_delete[0])
                print(duplicate_file_to_delete[0], "deleted.")
            except OSError:
                pass

print(f"{unique_ids}/112861 expected files exist in the directory.")

All duplicates removed and handled already.
96992/112861 expected files exist in the directory.
